In [1]:
from keras.layers import Embedding


In [2]:
from tensorflow.keras.layers import Embedding


In [2]:
import numpy as np
import time
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [8]:
data = pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
# Step 1: Load and preprocess the dataset
df = pd.read_csv('IMDB Dataset.csv')

# Map 'positive' to 1 and 'negative' to 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)# It ensures that only the most common 5000 words are considered for tokenization
tokenizer.fit_on_texts(df['review'])#This step essentially builds the word index (vocabulary) based on the text data in the 'review' column
X = tokenizer.texts_to_sequences(df['review'])
#This line converts the text data in the 'review' column into sequences of integers based on the vocabulary learned from the previous step.
X = pad_sequences(X, padding='pre', maxlen=100)
#the maxlen parameter specifies the maximum length of sequences
y = df['sentiment']
#This line extracts the target variable ('sentiment') from the DataFrame.

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
# Step 2: Define the deep neural network model architecture

embedding_dim = 128
#embedding_dim is set to 128, meaning each word will be represented by a dense vector of length 128.
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_dim))
#This line adds an Embedding layer to the model. The Embedding layer is responsible for converting integer-encoded words (represented by indices) into dense vectors of fixed size (the embedding dimension).
#input_dim=5000 specifies the size of the vocabulary, i.e., the maximum integer index that can be expected in the input data. This parameter should match the number of unique words in the vocabulary.
#output_dim=embedding_dim sets the dimensionality of the dense embedding vectors.
model.add(SpatialDropout1D(0.2))
#Spatial dropout is a variant of dropout that applies dropout specifically to the feature maps of the preceding layer.
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='sigmoid'))

In [13]:
# Step 3: Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Record start time
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
# Record end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")

# Step 5: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


In [10]:
# Step 6: Make predictions on new data (dynamic input)
while True:
    user_input = input("Enter a movie review: ")
    # Preprocess the user input
    input_sequence = tokenizer.texts_to_sequences([user_input])
    input_sequence = pad_sequences(input_sequence, padding='pre', maxlen=100)
    
    start_time = time.time()
    # Make prediction
    prediction = model.predict(input_sequence)[0][0]
    
    end_time = time.time()
    # Print prediction
    if prediction >= 0.5:
        print("Predicted sentiment: Positive")
    else:
        print("Predicted sentiment: Negative")
        
        # Calculate training time
    prediction_time = end_time - start_time
    print(f"Prediction time: {prediction_time:.2f} seconds")

Enter a movie review: Must watch movie for every aspirants. The movie based on real life story. The struggle faced by a main character showed beautiful in the movie which makes movie stand out. The story presented in a best way which makes you cry, laugh and enjoy at the same time. It's a struggle story of aspirants who want to achieve his success at any cost. It teaches us no matter from where you are, what's your condition you still have a chance to win. Don't lose hope. Everyone must watch the movie and get inspired to do something in your life. And achieve your goal. The Vikrant did the best job of his career potraiting a student with an atmost sincerity. Watch ylit you will never regret it.


NameError: name 'model' is not defined

Example:-

Movie:- 12th fail
Positive review:-

Must watch movie for every aspirants. The movie based on real life story. The struggle faced by a main character showed 
beautiful in the movie which makes movie stand out. The story presented in a best way which makes you cry, laugh and 
enjoy at the same time. It's a struggle story of aspirants who want to achieve his success at any cost. It teaches us no 
matter from where you are, what's your condition you still have a chance to win. Don't lose hope. Everyone must watch the 
movie and get inspired to do something in your life. And achieve your goal. The Vikrant did the best job of his career 
potraiting a student with an atmost sincerity. Watch ylit you will never regret it.

Movie:- Fast X
    
Negative review:
    
Enter a movie review: I can write the exact same review for the last Transformers, last Indiana Jones, last Star Wars movies, 
last FF or last Marvel movies etc... What happaned people? Something has happened in movie industry in late 2010's - 
2017 or 18. We cannot see any decent plot, any consistent story-telling or any sympathetic characters anymore. Day after 
day we have to endure more & more imbecilic movies with idiotic characters... new movies come out and this pattern never 
changes.  It's really not an easy job to be a movie-lover as of 2020's! :((